In [1]:
from openai.types.responses import (ResponseTextDeltaEvent, ResponseFunctionCallArgumentsDeltaEvent)
from agents import Agent, Runner
from agents import function_tool
import uuid
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# API 키는 환경 변수로 설정하세요. 소스 코드에 하드코딩하지 마세요.


In [3]:
data_storage = {}

In [4]:
@function_tool
def kaggle_list_datasets(keyword: str) -> str:
    
    """
    검색 쿼리에 맞는 Kaggle 데이터셋 목록을 조회합니다
    """
    from kaggle.api.kaggle_api_extended import KaggleApi

    try:
        # 데이터셋 검색
        api = KaggleApi()
        api.authenticate()
        datasets = api.dataset_list(search=keyword, sort_by="hottest")
        
        result = []
        count = 0
        for dataset in datasets:
            if count >= 10:
                break
            dataset_info = {
                "ref": dataset.ref,  # owner/dataset-name 형식
                "title": dataset.title,
            }
            
            # 안전하게 속성 추가
            if hasattr(dataset, 'size'):
                dataset_info["size"] = dataset.size
            if hasattr(dataset, 'lastUpdated'):
                dataset_info["last_updated"] = str(dataset.lastUpdated)
            if hasattr(dataset, 'downloadCount'):
                dataset_info["download_count"] = dataset.downloadCount
            if hasattr(dataset, 'voteCount'):
                dataset_info["vote_count"] = dataset.voteCount
            if hasattr(dataset, 'tags'):
                dataset_info["tags"] = dataset.tags
            if hasattr(dataset, 'usabilityRating'):
                dataset_info["usability_rating"] = dataset.usabilityRating
            
            result.append(dataset_info)
            count += 1
        
        return {
            "success": True,
            "count": len(result),
            "datasets": result
        }
    except Exception as e:
        return {
            "success": False,
            "message": f"데이터셋 목록 조회 실패: {str(e)}"
        }

In [5]:
@function_tool
def kaggle_check_and_download_dataset(dataset_ref: str, path: str = None) -> str:
    """
    데이터셋이 이미 다운로드되어 있는지 확인하고, 없으면 다운로드합니다.
    
    Args:
        dataset_ref: 다운로드할 데이터셋의 참조 (owner/dataset-name 형식)
        path: 데이터셋을 저장할 경로 (기본값: None, Kaggle API 기본 경로 사용)
    
    Returns:
        데이터셋 상태 및 경로 정보
    """
    import os
    from kaggle.api.kaggle_api_extended import KaggleApi
    
    try:
        api = KaggleApi()
        api.authenticate()
        
        # 데이터셋 정보 추출
        owner, dataset_name = dataset_ref.split('/')
        
        # 기본 Kaggle 다운로드 경로 또는 사용자 지정 경로
        if path is None:
            from pathlib import Path
            kaggle_dir = os.path.join(str(Path.home()), '.kaggle', 'datasets')
            dataset_dir = os.path.join(kaggle_dir, owner, dataset_name)
        else:
            dataset_dir = os.path.join(path, owner, dataset_name)
        
        # 데이터셋이 이미 존재하는지 확인
        if os.path.exists(dataset_dir) and len(os.listdir(dataset_dir)) > 0:
            # 데이터셋이 이미 존재함
            files = os.listdir(dataset_dir)
            data_storage[dataset_ref] = dataset_dir
            
            return {
                "success": True,
                "already_downloaded": True,
                "message": f"데이터셋 '{dataset_ref}'는 이미 다운로드되어 있습니다.",
                "path": dataset_dir,
                "files": files
            }
        else:
            # 데이터셋 다운로드
            api.dataset_download_files(dataset_ref, path=path, unzip=True)
            
            # 다운로드 후 경로 확인 및 저장
            if path is None:
                # 기본 경로에 다운로드된 경우
                dataset_dir = os.path.join(kaggle_dir, owner, dataset_name)
            
            if os.path.exists(dataset_dir):
                files = os.listdir(dataset_dir)
                data_storage[dataset_ref] = dataset_dir
                
                return {
                    "success": True,
                    "already_downloaded": False,
                    "message": f"데이터셋 '{dataset_ref}'를 성공적으로 다운로드했습니다.",
                    "path": dataset_dir,
                    "files": files
                }
            else:
                return {
                    "success": False,
                    "message": f"데이터셋 '{dataset_ref}'를 다운로드했지만 경로를 찾을 수 없습니다."
                }
    
    except Exception as e:
        return {
            "success": False,
            "message": f"데이터셋 다운로드 중 오류 발생: {str(e)}"
        }


In [6]:
@function_tool
def kaggle_download_dataset(dataset_ref: str) -> str:
    """
    Kaggle 데이터셋을 다운로드합니다
    """
    from kaggle.api.kaggle_api_extended import KaggleApi
    api = KaggleApi()
    api.authenticate()
    try:
        # Kaggle API로 데이터셋 다운로드
        api.dataset_download_files(dataset_ref, path=f"datasets/{dataset_ref}/", unzip=True)
        import os
        import pandas as pd
        
        # 다운로드 성공 시 처리
        dataset_path = f"datasets/{dataset_ref}/"
        
        # 디렉토리 내 파일 목록 가져오기
        files = os.listdir(dataset_path)
        
        # CSV 파일 찾기
        csv_files = [f for f in files if f.endswith('.csv')]
        
        if csv_files:
            # 첫 번째 CSV 파일 로드
            first_csv = os.path.join(dataset_path, csv_files[0])
            df = pd.read_csv(first_csv)
            
            # 데이터 저장소에 저장
            data_storage[dataset_ref] = df
            
            return {
                "success": True,
                "message": f"데이터셋 다운로드 및 로드 성공 : (저장소 key : {dataset_ref})",
                "file_loaded": csv_files[0],
                "rows": len(df),
                "columns": len(df.columns)
            }
        else:
            return {
                "success": True,
                "message": f"데이터셋 다운로드 성공했으나 CSV 파일이 없습니다: {dataset_ref}",
                "files_available": files
            }
    except Exception as e:
        return {
            "success": False,
            "message": f"데이터셋 다운로드 실패: {str(e)}"
        }

In [7]:
@function_tool
def load_csv_from_url(url: str, key: str) -> str:
    import pandas as pd
    """
    URL에서 CSV 파일을 읽어 Pandas DataFrame으로 변환하고 메모리에 저장
    
    Args:
        url (str): CSV 파일 주소
        key (str): 저장할 키
        
    Returns:
        str: 작업 결과 메시지
    """
    try:
        df = pd.read_csv(url)
        data_storage[key] = df
        return f"CSV 파일이 성공적으로 로드됨 (행: {len(df)}, 열: {len(df.columns)})"
    except Exception as e:
        return f"오류 발생: {str(e)}"
     

In [8]:
@function_tool
def show_df_stats(key: str) -> str:
    """저장된 데이터프레임의 기본 통계 정보 출력
    저장된 데이터 형태  (key) : Pandas DataFrame
    """
    if key not in data_storage:
        return "해당 URL의 CSV 파일을 찾을 수 없음"
    
    df = data_storage[key]
    return f"""
    행 수: {len(df)}
    열 수: {len(df.columns)}
    결측치 수: {df.isnull().sum().sum()}
    컬럼 목록: {df.columns.tolist()}
    컬럼 타입: {df.dtypes.to_dict()}
    기초 통계: {df.describe().to_dict()}
    """

In [9]:
@function_tool
def random_sample(key: str, n: int) -> str:
    """저장된 데이터프레임에서 랜덤 샘플 추출
    저장된 데이터 형태  (key) : Pandas DataFrame
    """
    if key not in data_storage:
        return "해당 URL의 CSV 파일을 찾을 수 없음"
    new_key = str(uuid.uuid4())
    df = data_storage[key]
    random_sample_df = df.sample(n)
    data_storage[new_key] = random_sample_df
    return f"랜덤 샘플 추출 완료: (저장소 key : {new_key})"

In [10]:
@function_tool
def prep_df(key: str, target_column: str) -> str:
    """저장된 데이터프레임을 머신러닝 모델에 적합한 형태로 전처리
    저장된 데이터 형태  (key) : Pandas DataFrame
    새로 저장할 데이터 형태 (new_key) : (X, y) 형태의 튜플
    """
    new_key = str(uuid.uuid4())
    df = data_storage[key]
    if target_column not in df.columns:
        return f"해당 컬럼 {target_column}이 존재하지 않습니다."
    df = df[df[target_column].notna()]
    
    # 실수형 데이터만 남기기
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df = df[numeric_cols]
    
    if target_column not in df.columns:
        return f"타겟 컬럼 {target_column}이 실수형 데이터가 아니어서 제거되었습니다."
    
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    data_storage[new_key] = (X, y)
    return f"데이터프레임 전처리 완료 (실수형 데이터만 포함): (저장소 key : {new_key})"

In [11]:
@function_tool
def visualize_tsne(key: str) -> str:
    """
    저장된 데이터프레임을 t-SNE로 시각화
    저장된 데이터 형태 (key) : (X, y) 형태의 튜플
    새로 저장할 데이터 형태 (new_key) : Plotly 객체
    """
    from utils import TSNEVisualizer
    import plotly.io as pio
    from IPython.display import HTML
    new_key = str(uuid.uuid4())
    
    # 렌더러 설정
    pio.renderers.default = "notebook"
    
    (X, y) = data_storage[key]
    fig = TSNEVisualizer.visualize(X, y)
    
    # to_html 메서드로 HTML 문자열 생성하여 표시
    html_str = fig.to_html(include_plotlyjs='cdn')
    display(HTML(html_str))
    
    data_storage[new_key] = fig
    
    return f"t-SNE 시각화 완료: (저장소 key : {new_key})"

In [12]:
@function_tool
def kaggle_check_and_download_dataset(dataset_ref: str, path: str = None) -> str:
    """
    데이터셋이 이미 다운로드되어 있는지 확인하고, 없으면 다운로드합니다.
    
    Args:
        dataset_ref: 다운로드할 데이터셋의 참조 (owner/dataset-name 형식)
        path: 데이터셋을 저장할 경로 (기본값: None, Kaggle API 기본 경로 사용)
    
    Returns:
        데이터셋 상태 및 경로 정보
    """
    import os
    from kaggle.api.kaggle_api_extended import KaggleApi
    
    try:
        api = KaggleApi()
        api.authenticate()
        
        # 데이터셋 정보 추출
        owner, dataset_name = dataset_ref.split('/')
        
        # 기본 Kaggle 다운로드 경로 또는 사용자 지정 경로
        if path is None:
            from pathlib import Path
            kaggle_dir = os.path.join(str(Path.home()), '.kaggle', 'datasets')
            dataset_dir = os.path.join(kaggle_dir, owner, dataset_name)
        else:
            dataset_dir = os.path.join(path, owner, dataset_name)
        
        # 데이터셋이 이미 존재하는지 확인
        if os.path.exists(dataset_dir) and len(os.listdir(dataset_dir)) > 0:
            # 데이터셋이 이미 존재함
            files = os.listdir(dataset_dir)
            data_storage[dataset_ref] = dataset_dir
            
            return {
                "success": True,
                "already_downloaded": True,
                "message": f"데이터셋 '{dataset_ref}'는 이미 다운로드되어 있습니다.",
                "path": dataset_dir,
                "files": files
            }
        else:
            # 데이터셋 다운로드
            api.dataset_download_files(dataset_ref, path=path, unzip=True)
            
            # 다운로드 후 경로 확인 및 저장
            if path is None:
                # 기본 경로에 다운로드된 경우
                dataset_dir = os.path.join(kaggle_dir, owner, dataset_name)
            
            if os.path.exists(dataset_dir):
                files = os.listdir(dataset_dir)
                data_storage[dataset_ref] = dataset_dir
                
                return {
                    "success": True,
                    "already_downloaded": False,
                    "message": f"데이터셋 '{dataset_ref}'를 성공적으로 다운로드했습니다.",
                    "path": dataset_dir,
                    "files": files
                }
            else:
                return {
                    "success": False,
                    "message": f"데이터셋 '{dataset_ref}'를 다운로드했지만 경로를 찾을 수 없습니다."
                }
    
    except Exception as e:
        return {
            "success": False,
            "message": f"데이터셋 다운로드 중 오류 발생: {str(e)}"
        }


In [13]:
@function_tool
def under_resample(key: str) -> str:
    """
    저장된 데이터프레임을 언더리샘플링
    저장된 데이터 형태  (key) : (X, y) 형태의 튜플
    새로 저장할 데이터 형태 (new_key) : (X, y) 형태의 튜플
    """
    from utils import ImbalancedDataAnalyzer
    new_key = str(uuid.uuid4())
    (X, y) = data_storage[key]
    analyzer = ImbalancedDataAnalyzer(X, y)
    X_resampled, y_resampled = analyzer.random_undersample()
    data_storage[new_key] = (X_resampled, y_resampled)
    return f"SMOTE 리샘플링 완료: (저장소 key : {new_key})"

In [14]:
@function_tool
def isolation_forest(key: str, contamination: float) -> str:
    """
    저장된 데이터프레임을 Isolation Forest로 분석
    저장된 데이터 형태  (key) : (X, y) 형태의 튜플
    새로 저장할 데이터 형태 (new_key) : (y, score) 형태의 튜플
    """
    from sklearn.ensemble import IsolationForest
    import plotly.express as px
    new_key = str(uuid.uuid4())
    
    (X, y) = data_storage[key]
    model = IsolationForest(contamination=contamination, max_samples=256)
    model.fit(X)
    score = model.score_samples(X)
    score = -1 * score  # 이부분임.
    fig = px.histogram(x=score, nbins=100, labels={'x':'Score'}, title="평가")
    fig.update_layout(width=600, height=400)
    fig.show()
    data_storage[new_key] = (y, score)
    return f"Isolation Forest 분석 완료: (Score 저장소 key : {new_key})"

In [15]:
@function_tool
def prec_rec_f1_curve(key: str):
    """
    저장된 데이터프레임을 Precision, Recall, F1 Score 곡선으로 시각화
    저장된 데이터 형태  (key) : (y, score) 형태의 튜플
    새로 저장할 데이터 형태 (new_key) : (precision, recall, f1, thresholds) 형태의 튜플
    """
    import numpy as np
    import plotly.graph_objects as go
    from sklearn.metrics import precision_recall_curve
    
    new_key = str(uuid.uuid4())
    y, score = data_storage[key]
    
    precision, recall, thresholds = precision_recall_curve(y, score, pos_label=1)
    f1 = 2 / (1/precision + 1/recall)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(precision, -1), mode='lines', name='precision'))
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(recall, -1), mode='lines', name='recall'))
    fig.add_trace(go.Scatter(x=thresholds, y=np.delete(f1, -1), mode='lines', name='f1'))
    fig.update_layout(title='Anomaly Score에 따른 Precision, Recall, F1 Score',
                      xaxis_title='Anomaly Score',
                      yaxis_title='Score',
                      legend_title='Score Type')
    fig.show()
    
    data_storage[new_key] = (precision, recall, f1, thresholds)

    return precision, recall, f1, thresholds


In [16]:
@function_tool
def show_tool_list() -> str:
    """
    현재 사용 가능한 도구 목록을 출력
    """
    tool_names = [
        "show_tool_list"
        "kaggle_list_datasets",
        "kaggle_download_dataset",
        "show_df_stats",
        "random_sample",
        "prep_df",
        "visualize_tsne",
        "under_resample",
        "isolation_forest",
        "prec_rec_f1_curve",
        
    ]
    return f"현재 사용 가능한 도구 목록: {', '.join(tool_names)}"



In [17]:
agent = Agent(
    name="Assistant",
    instructions=
        "너는 데이터 분석 전문가야"
        "가능한 한 항상 제공된 도구를 사용하는 것을 기억해야되."
        "너가 사용가능한 도구 목록을 먼저 출력해놓고 시작해. show_tool_list 도구를 사용하면 됨."
        "자신의 지식에 너무 의존하지 말고 대신 질문에 답하는 데 도움이 되는 도구를 사용하세요."
        "python 코드를 직접구현하는 방식을 사용하지 말고, 도구에 의존하세요."
        "t-SNE 시각화나 다른 시각화를 사용할 때 샘플 수가 적어 'perplexity must be less than n_samples' 등의 오류가 발생하면, "
        "해당 시각화 단계를 건너뛰고 사용자에게 샘플 수가 부족하여 시각화가 불가능하다고 알려주세요.",
    model="gpt-4o-mini",
    tools=[
        show_tool_list,
        kaggle_list_datasets,
        kaggle_download_dataset,
        show_df_stats,
        random_sample,
        prep_df,
        visualize_tsne,
        under_resample,
        isolation_forest,
        prec_rec_f1_curve,
    ]
)

In [18]:
# we do need to reinitialize our runner before re-executing
response = Runner.run_streamed(
    starting_agent=agent,
    input=
        "1. 사기 이상탐지 관련 데이터셋을 캐글에서 조회한후,"
        "2. 첫번째 데이터셋을 다운로드해줘."
        "3. 다운로드 받은 데이터셋에서 3000개 레코드만 랜덤 샘플링해줘."
        "4. 3000개 레코드의 기초 통계량을 출력해줘."
        "5. 앞에서 처리한 랜덤 샘플링한 3000개 레코드를 머신러닝 모델에 적합한 형태로 전처리해줘."
        "6. 전처리한 데이터를 TSNE 시각화해줘."
        "7. 전처리한 데이터를 SMOTE 리샘플링을 해줘."
        "8. SMOTE 리샘플링한 데이터를 TSNE 시각화해줘."
        "9. SMOTE 리샘플링한 데이터를 Isolation Forest 분석해줘."
        "10. Isolation Forest 분석한 데이터를 Precision, Recall, F1 Score 곡선으로 시각화해줘." 
)

async for event in response.stream_events():
    if event.type == "raw_response_event":
        if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent):
            # 도구 호출을 위한 스트리밍 매개변수
            print(event.data.delta, end="", flush=True)
        elif isinstance(event.data, ResponseTextDeltaEvent):
            # 스트리밍된 최종 응답 토큰
            print(event.data.delta, end="", flush=True)
    elif event.type == "agent_updated_stream_event":
        # 현재 사용 중인 에이전트 정보
        print(f"> 현재 에이전트: {event.new_agent.name}")
    elif event.type == "run_item_stream_event":
        # 사용자나 다운스트림 프로세스로 스트리밍할 정보를 포함하는 이벤트
        if event.name == "tool_called":
            # 모든 도구 토큰이 스트리밍된 후 전체 도구 호출 정보
            print()
            print(f"> 도구 호출됨, 이름: {event.item.raw_item.name}")
            print(f"> 도구 호출됨, 인자: {event.item.raw_item.arguments}")
        elif event.name == "tool_output":
            # 도구 실행 결과
            print(f"> 도구 출력: {event.item.raw_item['output']}")

> 현재 에이전트: Assistant


Error streaming response: Project `proj_9jPb3KpbTdfDPtpFQRloSjOo` does not have access to model `gpt-4o-mini-2024-07-18`


APIError: Project `proj_9jPb3KpbTdfDPtpFQRloSjOo` does not have access to model `gpt-4o-mini-2024-07-18`

In [16]:
# data_storage.keys()

In [17]:
# data_storage['e979a416-372c-4766-8a37-8189f4ba7d49']